We recommend you **Restart and Clear Output** the notebook kernel so there is sufficient memory for below.

import pandas as pd

df = pd.read_csv('../../data/tute_data/sample_data.csv')

## Plotting a Geospatial map using `folium` and `geopandas`
You'll need to `pip3 install folium geopandas` for this section.

Some terminology before we continue:
- Map Tiles / Tile Providers: The underlying map "style" (i.e Google Maps vs Open Street View)

Let's start off with `folium`.

import folium

# initialise a map
_map = folium.Map(tiles="Stamen Terrain", zoom_start=10)

# save map
_map.save('../../plots/map.html')

# show map
_map

Since `folium` generates interactive maps, you will need to save static versions for your report.

The easiest way is to open the saved `html` plot and take a **good and high-quality screenshot**. Make sure to take into account the title, legend, and font size.

## Where to go from here
We've 

## Geospatial Inferences
- More pickups around central Manhattan, with more dropoffs in the surrounding bouroughs.
- Pickup location are easily divided into "hubs" (i.e Manhattan, Aiport, etc).
- Dropoffs seem to be scattered across the map.

**IMPORTANT:** The above is at most *describing* the plot. Your project will require *analysis* and *research* on top of describing a plot. That is:
- *Why might there be more pickups around central Manhattan?*
- *Is there an explanation surrounding the "hubs"?*
- *Why are dropoffs scattered across the map?*

As a suggestion, have less description and more analysis. Your visualisation should ensure that it can be easily interpreted and visible (i.e suitable font size, colour, alpha, legend, etc.)

# Lab 2 Overview
## First Half
More Visualisations:
- Map Clusters
- Heatmaps (GIS)
- HexBins vs SquareBins 
- Choropleths

Descriptive Statistics:
- Histograms and looking at the distribution shape.
- How to "quick plot" dataframes using `pandas`.
- Some methods of determining bin sizes.

## Second Half
Revision:
- Any code related questions for Python
- (Windows 10 Users) Installing WSL2 (Ubuntu 20.04) for a clean environment.

Advanced Content:
- Introduction to `PySpark`.

_________________


# Visualisations
- Clustering points
- Binned plots
- Good visualizations vs Bad visualisations


## Personal Checklist for Visualisations and Dashboards:
1. Your visualisation needs to tell a story.
2. It should be interpretable without being overly verbose.
3. The scale and axis need to make sense (and you can assume the reader knows the difference between a normal scale vs log scale).
4. The choice of visualisation needs to make sense:
    - Line plot vs Bar chart with non-numerical categories
    - Map plot with points vs clusters for each location
    - Scatterplot vs Histogram plot to see distribution
    - etc
5. Choice of colour scheme / alpha / size need to be easy on the eyes.

At the end of the day, even if you think your visualisation is "pretty" or "beautiful", if a reader cannot understand it, then it is not a good visualisation

In [ ]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# read in the data
df = pd.read_feather("../data/lab_specific/sample.feather").drop('index', axis=1)

df.tail()

## Clustering
- Excellent tool when visualising geospatial coordinates
- Aggregates certain "areas" to make distributions far easier to interpret

Questions:
- *How is this worse/better than the previous scatter plot?*
- *Does this plot give us any useful information?*

In [ ]:
import folium
from folium.plugins import FastMarkerCluster

# lat, long
COORDS = ['pickup_latitude', 'pickup_longitude']

# create an interactive geospatial graph
pickups_cluster = folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)

# use a built-in clustering algorithm to apply markers for hotspots
pickups_cluster.add_child(FastMarkerCluster(data=df[COORDS].values))

# visualize the plot 
pickups_cluster.save('../plots/foliumFastCluster.html')
pickups_cluster

## Heatmaps
- Similar to clustering with respect to visualising geospatial coordinates.
- Allows for a more continuous interpretation of the map, rather than clusters.

Questions:
- *In terms of pickups, why might we prefer or not prefer this over a cluster map?*

In [ ]:
from folium.plugins import HeatMap

pickups_heatmap = folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)
pickups_heatmap.add_child(HeatMap(df[COORDS].values, radius=10))

pickups_heatmap.save('../plots/foliumPickupHeatmap.html')
pickups_heatmap

In [ ]:
dropoffs_heatmap = folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)
dropoffs_heatmap.add_child(HeatMap(df[['dropoff_latitude', 'dropoff_longitude']].values, radius=10))

dropoffs_heatmap.save('../plots/foliumDropoffHeatmap.html')
dropoffs_heatmap

## Hex Binned Plots
- An extension of heatmaps that discretises the points.
- Gives a greater granularity to the map when looking at geospatial coordinates.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import save, reset_output, output_notebook

TILE = get_provider("STAMEN_TERRAIN_RETINA")

reset_output()
output_notebook()
# note below that it says "BokehJS 1.4.0 successfully loaded."

In [ ]:
df[COORDS].describe()

In [ ]:
df.dropna(inplace=True)

# create left and right boundaries for the x and y axis respectively
xRange = [-74, -73.9]
yRange = [40.5, 40.9]

def lat2mercer(coords):
    k = 6378137
    converted = list()
    for lat in coords:
        converted.append(np.log(np.tan((90 + lat) * np.pi/360.0)) * k)
    return converted

def lon2mercer(coords):
    k = 6378137
    converted = list()
    for lon in coords:
        converted.append(lon * (k * np.pi/180.0))
    return converted

# convert to mercer
df['pickupX'] = df['pickup_longitude'].apply(lambda x: lon2mercer([x])[0])
df['pickupY'] = df['pickup_latitude'].apply(lambda x: lat2mercer([x])[0])

In [ ]:
# create bokeh figure, where x_range and y_range are in mercer
p = figure(x_range=lon2mercer(xRange), y_range=lat2mercer(yRange),
           x_axis_type="mercator", y_axis_type="mercator")
# add map tile
p.add_tile(TILE)
# change title
p.title.text = "Hex-Binned Pickups in NYC"

# add hexbin (size param is the bin size - more about it next week)
p.hexbin(x=df['pickupX'], y=df['pickupY'], size=250)

show(p)
save(p, '../plots/bokehPickupHexBinned.html')

Changing the color and adding color bar for a better visualization

In [ ]:
from bokeh.models import ColorBar, LinearColorMapper
from bokeh.palettes import all_palettes

# create bokeh figure, where x_range and y_range are in mercer
p = figure(x_range=lon2mercer(xRange), y_range=lat2mercer(yRange),
           x_axis_type="mercator", y_axis_type="mercator")
# add map tile
p.add_tile(TILE)
# change title
p.title.text = "Hex-Binned Pickups in NYC"

palette = all_palettes['Magma'][256][::-1]
color_mapper = LinearColorMapper(palette=palette, low=1, high=1449)
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12)
r, bins = p.hexbin(x=df['pickupX'], y=df['pickupY'], size=250, palette=palette)

p.add_layout(color_bar, 'right')

show(p)
save(p, '../plots/bokehPickupHexBinned.html')

An alternative using matplotlib. Although fast, it's not useful without the underlying geospatial features.

In [ ]:
import matplotlib.pyplot as plt

test = df.dropna()
plt.figure(figsize=(10,10))

plt.hexbin(x = test[COORDS[1]], y= test[COORDS[0]],
           gridsize=100, bins='log', cmap='inferno')

plt.show()

_________________


## Shapefile Overlays
- **NOTE: This only applies on the more recent datasets that use zones over coordinates**

Requirements:
- `geopandas`

Shapefile Links:
- https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip
- https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

Installation:
- MacOS and Linux users use `pip install geopandas` or equivalent.
- Windows users may need to visit another guide if there are issues (see manual installation).

Manual:
1. Visit https://www.lfd.uci.edu/~gohlke/pythonlibs/
2. You will need 2 different `.whl` (wheel) files. These are `GDAL` and `Fiona`.
3. Find both of them and download the version **corresponding to your device OS and Python**. For example, a 64-bit Windows 10 device running Python 3.8.X will need to download this file for `GDAL`. Repeat this for `Fiona` as well. (For your convenience, a fresh installation of Ubuntu 20.04 will yield Python 3.8.3, and most modern devices all run on 64-bit architecture. I have provided these in the `geopandas_dependencies` folder)

![gdal](./geopandas_dependencies/tute.PNG)

4. Once they are downloaded, you will need to open up command prompt and `cd` into the directory. For example, if my `.whl` files are in the `geopandas_dependencies` folder, the command I would use is:
    - `cd C:\Users\USERNAME\Documents\GitHub\MAST30034Repo\workbook\geopandas_dependencies`
5. Install the dependencies in this order:
    - `pip install GDAL-3.1.2-cp37-cp37m-win_amd64.whl` (or the corresponding file you downloaded)
    - `pip install Fiona-1.8.13-cp37-cp37m-win_amd64.whl`
6. Run `pip install geopandas` and it should now work

In [ ]:
df = pd.read_csv("../data/lab_specific/yellow_tripdata_2019-01.csv")
df.groupby('PULocationID')['payment_type'].count().reset_index().sort_values('payment_type', ascending=False)

In [ ]:
import geopandas as gpd

# sf stands for shape file
sf = gpd.read_file("../data/taxi_zones/taxi_zones.shp")
zone = pd.read_csv("../data/taxi_zones/taxi+_zone_lookup.csv")

# Convert the geometry shaape to to latitude and longitude
# Please attribute this if you are using it
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [ ]:
sf

Common Conventions:
- `gdf` stands for GeoDataFrame.
- Now, we need to create a `gdf` with our current dataframe and shapefile data.

From `df`, we join using `PULocationID` to match from `sf`'s `LocationID`

In [ ]:
sf.head()

In [ ]:
gdf = gpd.GeoDataFrame(pd.merge(df, sf, left_on='PULocationID', right_on='LocationID')).drop('PULocationID',axis=1)
gdf.sample(2)

Now, lets try plot this with `folium`. There are two ways to approach this kind of task:
- Plot all the data!
- Aggregate *before* plotting data!

It's up to you to decide which to use...

Specifically with `folium`, you will need to create a GeoJSON format. You can view more information using the documentation.

In [ ]:
geoJSON = gdf[['LocationID','geometry']].drop_duplicates('LocationID').to_json()

This plot is just shapefiles based on the locations and **does not** show any analysis.

Use this as a reference on how you can plot shapefiles...

In [ ]:
m = folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on how to plot aggregated data.
m.add_child(folium.Choropleth(
    geo_data=geoJSON,
    name='choropleth',
))

m.save('../plots/foliumChoroplethMap.html')
m

In [ ]:
import json

# an example of what the geoJSON looks like
json.loads(geoJSON)

In [ ]:
gdf.loc[gdf['total_amount'] < 0]

In [ ]:
gdf[['LocationID','total_amount']].groupby('LocationID').sum().reset_index()

In [ ]:
m_trip_distance = folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on more information on how to plot aggregated data.
folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=gdf, # data source
    columns=['LocationID','total_amount'], # the columns required
    key_on='properties.LocationID', # this is from the geoJSON's properties
    fill_color='OrRd', # color scheme
    fill_opacity=0.9,
    line_opacity=0.5,
    legend_name='Trips' # legend title
).add_to(m_trip_distance)

m_trip_distance.save('../plots/foliumChoroplethMapTrips.html')
m_trip_distance

_________________


##  Descriptive Statistics
- Hopefully nothing here is *new* to you...
- This tutorial is more about efficiently getting there and some food for thought.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# read in the data
df = pd.read_feather("../data/lab_specific/sample.feather").dropna().drop('index', axis=1)

# describe our data
df.describe()

- Remember that not all the data should be interpreted as purely numerical...
- There may be conclusions you can draw *by coincidence* if you incorrectly assume data types!
- For example:
    - `longitude` and `latitude` should be interpreted as geospatial coordinates
    - `payment_type` is a discrete category of payment types
    - `trip_distance` is non-linear (not a straight line from A to B), but we have no further data on it. It is also in **miles**.
- To avoid misinterpreting the attributes, refer to the data dictionary provided on the TLC website.

## Scatterplot plots for Fare / Distance

In [ ]:
df['RatecodeID'].value_counts()

In [ ]:
df[['fare_amount', 'trip_distance']].plot.scatter(x='fare_amount',
                                                  y='trip_distance')

plt.show()

General Inference:
- We can visually see that the relationship is relatively linear as you'd expect (more distance generally means more money)
- A fair number of outliers, notably around the 0 distance axis and 0 cost axis. Why might this be the case?
- Why are there negative values?

## Histogram plots for Trip Distance

In [ ]:
sns.distplot(df['fare_amount'], bins=30)

plt.show()

- I guess we can kind of see most fares are between 0 - 100.
- Hard to tell where the main distribution is spread around.
- We can enhance this using a log transformation.

In [ ]:
from numpy import log, sqrt

# apply a log transformation for all x non-zero x points, else 0
def logify(x):
    return log(x) if x else 0

sns.distplot(df['fare_amount'].apply(logify), bins=50)
plt.show()

A one-line `lambda` alternative.

In [ ]:
sns.distplot(df['fare_amount'].apply(lambda x: log(x) if x else 0), bins=50)
plt.show()

- Take a log transformation to visually see the distribution
- Now we see most the values fall under `exp(x)`, majority between `$7` - `$55` (`exp(2)` - `exp(55)`)

How about the distribution for trips under 15 miles?

In [ ]:
data = df.loc[df['trip_distance'] <= 15, 'trip_distance']

sns.distplot(data, bins=30)
plt.show()

Instant Insights:
- Distribution is positive skewed
- Fare distances are predominantly very short, between 1-3 miles
- There seems to be a fare number of outliers for trips > 15 miles
- Perhaps we should do a correlation check. Recall that we had negative fares for some reason...

In [ ]:
# pearson (by default) correlation table for distance and fare amount
df[['trip_distance','fare_amount']].corr(method='pearson')

## Correlation Heatmap

In [ ]:
sns.heatmap(df.corr())
# wow that's easy...

plt.show()

Things to take note of:
- `trip_distance` highly correlates with high tips, tolls and overall trip amount
- `payment_type` seems to have some form of negative correlation with `tip_amount`. Must be careful as this is a discrete category.
- As you can see here, having `VendorID` as an feature is misleading, you should take any factor out when conducting Pearson Correlation.


In [ ]:
CORR_COLS = ["passenger_count", "trip_distance", "fare_amount", "extra", 
 "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge", "total_amount"]

df[CORR_COLS].corr()

In [ ]:
sns.heatmap(df[CORR_COLS].corr())
plt.show()

- If you're interested in calculating correlation between nominal and continuous data, here's a [great explanation](https://stats.stackexchange.com/questions/119835/correlation-between-a-nominal-iv-and-a-continuous-dv-variable/124618#124618).   
- Remember, you need to refer back to the data dictionary as well as the fare page: https://www1.nyc.gov/site/tlc/passengers/taxi-fare.page

- You should especially take note of the fare page if you're looking to see how `RatecodeID` plays a role on the fare.

## Binning Methods
Sturges:  
- $bins = ceil(log_2(n)) + 1$

Rice:
- $bins = 2\times\sqrt[3]{n}$

Scott:
- $bins = \frac{max - min}{3.5\times \frac{SD}{\sqrt[3]{n}}}$

Freedman:
- $bins = \frac{max - min}{2\times \frac{IQR}{\sqrt[3]{n}}}$

Square:
- $bins = \sqrt{n}$

(Source: https://www.answerminer.com/blog/binning-guide-ideal-histogram)

In [ ]:
# dataframes method that may be of use
MAX = df['fare_amount'].max()
MIN = df['fare_amount'].min()
SD = df['fare_amount'].std()
IQR = df['fare_amount'].quantile()
N = len(df)

Functions that implement specific binning methods. Feel free to use them if you would like so long as they are attributed.

In [ ]:
from numpy import log, log2

def sturges(x):
    return int(log2(x)) + 1

def rice(x):
    return int(2 * x ** (1/3))

def scott(large, small, sd, x):
    return int((large - small) / (3.5 * (sd/x ** (1/3))))

def freedman(large, small, iqr, x):
    return int((large - small) / (2 * (iqr/x ** (1/3))))
    
def square(x):
    return int(sqrt(x))

def logify(x):
    return log(x) if x else 0

In [ ]:
fig1 = sns.distplot(df['fare_amount'], bins=sturges(N))
plt.title("Sturges Binnings")
plt.show()

fig2 = sns.distplot(df['fare_amount'], bins=rice(N))
plt.title("Rice Binnings")
plt.show()

fig3 = sns.distplot(df['fare_amount'], bins=scott(MAX, MIN, SD, N))
plt.title("Scott Binnings")
plt.show()

fig4 = sns.distplot(df['fare_amount'], bins=freedman(MAX, MIN, IQR, N))
plt.title("Freedman Binnings")
plt.show()

fig5 = sns.distplot(df['fare_amount'], bins=square(N))
plt.title("Square Binnings")
plt.show()

In [ ]:
fig1 = sns.distplot(df['fare_amount'].apply(logify), bins=sturges(N))
plt.title("log Sturges Binnings")
plt.show()

fig2 = sns.distplot(df['fare_amount'].apply(logify), bins=rice(N))
plt.title("log Rice Binnings")
plt.show()

fig3 = sns.distplot(df['fare_amount'].apply(logify), bins=scott(MAX, MIN, SD, N))
plt.title("log Scott Binnings")
plt.show()

fig4 = sns.distplot(df['fare_amount'].apply(logify), bins=freedman(MAX, MIN, IQR, N))
plt.title("log Freedman Binnings")
plt.show()

fig5 = sns.distplot(df['fare_amount'].apply(logify), bins=square(N))
plt.title("log Square Binnings")
plt.show()

- Rice's method for binning looks good **for this plot**
- Not always the case, keep that in mind...

_________________


## Feature Engineering?
- We want to see if the the profitability of zones remains consistent with respect to hour of day, day of week and pickup location. The distribution of profitable zones should be similar across all years.
- How is a zone profitable? Frequency of trips? Duration of trips? Best "earners"? 

- You could create your own feature and scale it accordingly. Perhaps the expected dollar per minute + possible tolls scaled by the expected frequency of trips might be a good start.

- Just remember that trip frequency $\approx$ taxi demand in a zone (you don't know the number of taxis in a zone at the time)

- Additionally, variable rate fares exist: _"50 cents per 1/5 mile when travelling above 12mph OR 50 cents per 60 seconds in slow traffic or when the vehicle is stopped."_

- Profit rates might assume crude approximations degrading into linear distance / constant velocity / etc

__________________

# Pre-Requisite Tasks for the Apache Spark tutorial

### WSL Environment for Windows 10
Refer to this guide to get a native Linux terminal in Windows 10:
- https://github.com/akiratwang/COMP20003-Setting-Up
- Ignore all the `C` related parts, just get Ubuntu installed.

### Apache Spark 3.0 (PySpark) Installation
- Visit `MAST30034/advanced_tutorials/Spark%20Installation.ipynb`

## Today
### Apache Spark 3.0 (PySpark and Spark SQL) Tutorial
- Visit `MAST30034/advanced_tutorials/Spark%20Tutorial.ipynb`